# Import libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
! pip install folium==0.5.0
import folium # map rendering library

     |████████████████████████████████| 79 kB 8.4 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=380ab4edf408084da6f855517b8a55c4c0d737b3b6d9f8fc56b32112abccf7d0
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


In [4]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [75]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [51]:
from sklearn.cluster import KMeans

In [48]:
import numpy as np

# Define functions

In [207]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [208]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [209]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# TASK 1 - Generate main dataframe

In [5]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')

#### parse wikitable

In [6]:
table_data = []
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    table_data.append([data[0].string.rstrip(), data[1].string.rstrip(),data[2].string.rstrip()])

#### generate dataframe

In [7]:
columns = ['PostalCode','Borough','Neighborhood']
main_df = pd.DataFrame(table_data, columns = columns)

#### remove Not assigned borough

In [8]:
clean_borough = main_df.drop(main_df[main_df.Borough == 'Not assigned'].index)

#### check if there is any Not assigned Neighborhood

In [9]:
clean_borough[clean_borough.Neighborhood == 'Not assigned']

,PostalCode,Borough,Neighborhood


#### there aren't Not assigned Neighborhoods

In [203]:
# shape of prepared dataframe
clean_borough.shape

(103, 3)

# Task 2 - add coodinates to dataframe

In [204]:
#download provided file
! wget "https://cocl.us/Geospatial_data"

--2021-02-16 19:47:33--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.63.96.176, 169.63.96.194
Connecting to cocl.us (cocl.us)|169.63.96.176|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-02-16 19:47:34--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-02-16 19:47:35--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following

In [12]:
#read csv into dataframe
geo_data = pd.read_csv('Geospatial_data')

In [13]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
#rename column to be able to merge dataframes
geo_data.rename(columns ={'Postal Code':'PostalCode'}, inplace = True)

In [15]:
#merge dataframes
working_df = pd.merge(clean_borough, geo_data, on = 'PostalCode')

In [16]:
working_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [205]:
working_df.shape

(103, 5)

# TASK 3 - Clustering

#### get Toronto coordinates

In [17]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
# create map of Toronoto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(working_df['Latitude'], working_df['Longitude'], working_df['Borough'], working_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Explore borough

### Notrh York is a testing borough

In [206]:
#north york dataframe
northyork_data = working_df[working_df['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


#### get North York coordinates

In [20]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [21]:
# create map of North York using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

#### Define Foursquare Credentials and Version


In [259]:
# The code was removed by Watson Studio for sharing.

## Explore one neighborhood

In [23]:
northyork_data.loc[1, 'Neighborhood']

'Victoria Village'

In [24]:
neighborhood_latitude = northyork_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = northyork_data.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = northyork_data.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Victoria Village are 43.725882299999995, -79.31557159999998.


### Get Foursquare data for 'Victoria Village'

In [231]:
#set radius and limit
LIMIT = 100
RADIUS = 700

In [232]:
#generate url
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude,ACCESS_TOKEN, VERSION, neighborhood_name, RADIUS, LIMIT)

In [233]:
#request data
results = requests.get(url).json()

In [234]:
#get venues
venues = results['response']['venues']

In [235]:
#create venues dataframe
nearby_venues = pd.json_normalize(venues) 

In [236]:
nearby_venues.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.postalCode,location.city,location.state,location.address,location.crossStreet
0,4c633acb86b6be9a61268e34,Victoria Village Arena,"[{'id': '4bf58dd8d48988d185941735', 'name': 'H...",v-1613506367,False,43.723481,-79.315635,"[{'label': 'display', 'lat': 43.72348055545508...",267,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN
1,5a02064f50a6f005378ac610,Victoria Village,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",v-1613506367,False,43.726322,-79.305770,"[{'label': 'display', 'lat': 43.726322, 'lng':...",790,CA,Canada,"[Toronto ON M4A 1T6, Canada]",M4A 1T6,Toronto,ON,NaN,NaN
2,53a733bd498ef6698c42b6da,1684 Victoria Park Ave.,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",v-1613506367,False,43.731100,-79.314300,"[{'label': 'display', 'lat': 43.7311, 'lng': -...",589,CA,Canada,"[1684 Victoria Park Ave. (at Draycott Dr.), Sc...",NaN,Scarborough,ON,1684 Victoria Park Ave.,at Draycott Dr.


In [237]:
# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Victoria Village Arena,Hockey Arena,43.723481,-79.315635
1,Victoria Village,Neighborhood,43.726322,-79.305770
2,1684 Victoria Park Ave.,Residential Building (Apartment / Condo),43.731100,-79.314300


## Explore all North York venues

In [238]:
northyork_venues = getNearbyVenues(names=northyork_data['Neighborhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [239]:
#group venues by Neighborhood
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Don Mills,27,27,27,27,27,27
Downsview,14,14,14,14,14,14
"Fairview, Henry Farm, Oriole",60,60,60,60,60,60
Glencairn,5,5,5,5,5,5
Hillcrest Village,5,5,5,5,5,5
Humber Summit,1,1,1,1,1,1


In [240]:
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 98 uniques categories.


## Analyze Each Neighborhood

In [241]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [242]:
northyork_onehot.shape

(238, 99)

#### group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [243]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()

In [244]:
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.095238,...,0.000000,0.047619,0.047619,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.040000,0.000000,0.08,0.000000,0.040000,0.000000,0.000000,0.000000
3,Don Mills,0.000000,0.000000,0.000000,0.037037,0.000000,0.037037,0.000000,0.000000,0.000000,...,0.000000,0.037037,0.037037,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Downsview,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.071429,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Fairview, Henry Farm, Oriole",0.016667,0.000000,0.016667,0.000000,0.000000,0.016667,0.000000,0.033333,0.033333,...,0.000000,0.000000,0.000000,0.016667,0.00,0.016667,0.016667,0.000000,0.016667,0.000000
6,Glencairn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,Hillcrest Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Humber Summit,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Humberlea, Emery",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [245]:
northyork_grouped.shape

(20, 99)

#### print each neighborhood along with the top 5 most common venues

In [246]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0    Coffee Shop  0.10
1           Bank  0.10
2       Pharmacy  0.05
3  Shopping Mall  0.05
4  Deli / Bodega  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4               Lounge  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.08
1         Coffee Shop  0.08
2     Thai Restaurant  0.08
3  Italian Restaurant  0.08
4            Boutique  0.04


----Don Mills----
                 venue  freq
0                  Gym  0.11
1       Clothing Store  0.07
2          Coffee Shop  0.07
3  Japanese Restaurant  0.07
4           Restaurant  0.07


----Downsview----
                  venue  freq
0                  Park  0.14
1                  Bank  0.07
2  Gym / Fitness Center  0.07
3      Business Service  0.07
4          Liquor Store  0.07


#### create the new dataframe and display the top 10 venues for each neighborhood.

In [247]:
num_top_venues = 12

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gas Station,Sandwich Place,Chinese Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Intersection,Bridal Shop,Pharmacy,Pizza Place,Ice Cream Shop
1,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Thai Restaurant,Coffee Shop,Sandwich Place,Pub,Grocery Store,Café,Butcher,Fast Food Restaurant,Indian Restaurant,Comfort Food Restaurant,Boutique
3,Don Mills,Gym,Clothing Store,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Sandwich Place,Caribbean Restaurant,Café,Italian Restaurant,Bike Shop,Dim Sum Restaurant
4,Downsview,Park,Bank,Gym / Fitness Center,Airport,Liquor Store,Food Truck,Hotel,Athletics & Sports,Grocery Store,Business Service,Snack Place,Shopping Mall


## Cluster Neighborhoods


#### Run _k_-means to cluster the neighborhood into 7 clusters.

In [248]:
# set number of clusters
kclusters = 7

horthyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(horthyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 3], dtype=int32)

In [249]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [250]:
northyork_merged = northyork_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

northyork_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,6,Park,Food & Drink Shop,Hotel,Department Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,5,French Restaurant,Coffee Shop,Hockey Arena,Portuguese Restaurant,Vietnamese Restaurant,Department Store,Chocolate Shop,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Accessories Store,Supermarket,Café,Caribbean Restaurant,Chinese Restaurant
3,M3B,North York,Don Mills,43.745906,-79.352188,0,Gym,Clothing Store,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Sandwich Place,Caribbean Restaurant,Café,Italian Restaurant,Bike Shop,Dim Sum Restaurant
4,M6B,North York,Glencairn,43.709577,-79.445073,0,Park,Pizza Place,Bakery,Pub,Japanese Restaurant,Vietnamese Restaurant,Deli / Bodega,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping


In [251]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [252]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
2,North York,0,Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Accessories Store,Supermarket,Café,Caribbean Restaurant,Chinese Restaurant
3,North York,0,Gym,Clothing Store,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Sandwich Place,Caribbean Restaurant,Café,Italian Restaurant,Bike Shop,Dim Sum Restaurant
4,North York,0,Park,Pizza Place,Bakery,Pub,Japanese Restaurant,Vietnamese Restaurant,Deli / Bodega,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
5,North York,0,Gym,Clothing Store,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Sandwich Place,Caribbean Restaurant,Café,Italian Restaurant,Bike Shop,Dim Sum Restaurant
6,North York,0,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Pool,Dog Run,Vietnamese Restaurant,Deli / Bodega,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
7,North York,0,Coffee Shop,Bank,Gas Station,Sandwich Place,Chinese Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Intersection,Bridal Shop,Pharmacy,Pizza Place,Ice Cream Shop
8,North York,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Japanese Restaurant,Food Court,Juice Bar,Bank,Bakery,Accessories Store,Electronics Store,Bus Station
9,North York,0,Vietnamese Restaurant,Bar,Coffee Shop,Massage Studio,Metro Station,Miscellaneous Shop,Caribbean Restaurant,Athletics & Sports,Dim Sum Restaurant,American Restaurant,Airport,Comfort Food Restaurant
10,North York,0,Café,Bank,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
11,North York,0,Park,Bank,Gym / Fitness Center,Airport,Liquor Store,Food Truck,Hotel,Athletics & Sports,Grocery Store,Business Service,Snack Place,Shopping Mall


#### Cluster 2

In [253]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
16,North York,1,Park,Camera Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
22,North York,1,Park,Convenience Store,Camera Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store


#### Cluster 3

In [254]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
15,North York,2,Intersection,Vietnamese Restaurant,Dim Sum Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


#### Cluster 4

In [255]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
19,North York,3,Baseball Field,Vietnamese Restaurant,Diner,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


#### Cluster 5

In [256]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 4, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
12,North York,4,Martial Arts School,Vietnamese Restaurant,Caribbean Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


#### Cluster 6

In [257]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 5, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
1,North York,5,French Restaurant,Coffee Shop,Hockey Arena,Portuguese Restaurant,Vietnamese Restaurant,Department Store,Chocolate Shop,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop


#### Cluster 7

In [258]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 6, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
0,North York,6,Park,Food & Drink Shop,Hotel,Department Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
